In [4]:
# -*- coding: UTF-8 -*-
# 利用鸢尾花数据集，实现前向传播、反向传播，可视化loss曲线

# 导入所需模块
import tensorflow as tf
from sklearn import datasets
from matplotlib import pyplot as plt
import numpy as np

# 导入数据，分别为输入特征和标签
x_data = datasets.load_iris().data
y_data = datasets.load_iris().target

# 随机打乱数据（因为原始数据是顺序的，顺序不打乱会影响准确率）
# seed: 随机数种子，是一个整数，当设置之后，每次生成的随机数都一样（为方便教学，以保每位同学结果一致）
np.random.seed(116)  # 使用相同的seed，保证输入特征和标签一一对应
np.random.shuffle(x_data) # make it mass
np.random.seed(116)
np.random.shuffle(y_data)
tf.random.set_seed(116) 

# 将打乱后的数据集分割为训练集和测试集，训练集为前120行，测试集为后30行
x_train = x_data[:-30] # last 30
y_train = y_data[:-30]
x_test = x_data[-30:]
y_test = y_data[-30:]

# 转换x的数据类型，否则后面矩阵相乘时会因数据类型不一致报错
# same type
x_train = tf.cast(x_train, tf.float32)
x_test = tf.cast(x_test, tf.float32)

# from_tensor_slices函数使输入特征和标签值一一对应。（把数据集分批次，每个批次batch组数据）
train_db = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
# every 32 is one batch !!!!
test_db = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 生成神经网络的参数，4个输入特征故，输入层为4个输入节点；因为3分类，故输出层为3个神经元
# 用tf.Variable()标记参数可训练
# 使用seed使每次生成的随机数相同（方便教学，使大家结果都一致，在现实使用时不写seed）
# input: 4 features
# output: 3 classes
w1 = tf.Variable(tf.random.truncated_normal([4, 3], stddev=0.1, seed=1))
b1 = tf.Variable(tf.random.truncated_normal([3], stddev=0.1, seed=1))

lr = 0.1  # 学习率为0.1
train_loss_results = []  # 将每轮的loss记录在此列表中，为后续画loss曲线提供数据
test_acc = []  # 将每轮的acc记录在此列表中，为后续画acc曲线提供数据
epoch = 500  # 循环500轮
loss_all = 0  # 每轮分4个step，loss_all记录四个step生成的4个loss的和

# 训练部分
for epoch in range(epoch):  #数据集级别的循环，每个epoch循环一次数据集
    for step, (x_train, y_train) in enumerate(train_db):  #batch级别的循环 ，每个step循环一个batch
        with tf.GradientTape() as tape:  # with结构记录梯度信息
            y = tf.matmul(x_train, w1) + b1  # 神经网络乘加运算
            # trans y to one-hot
            y = tf.nn.softmax(y)  # 使输出y符合概率分布（此操作后与独热码同量级，可相减求loss）
            y_ = tf.one_hot(y_train, depth=3)  # 将标签值转换为独热码格式，方便计算loss和accuracy
            # get loss
            # y(one-hot) - y value (softmax)
            loss = tf.reduce_mean(tf.square(y_ - y))  # 采用均方误差损失函数mse = mean(sum(y-out)^2)
            # tansfer into np
            loss_all += loss.numpy()  # 将每个step计算出的loss累加，为后续求loss平均值提供数据，这样计算的loss更准确
        # 计算loss对各个参数的梯度
        grads = tape.gradient(loss, [w1, b1])

        # 实现梯度更新 w1 = w1 - lr * w1_grad    b = b - lr * b_grad
        w1.assign_sub(lr * grads[0])  # 参数w1自更新
        b1.assign_sub(lr * grads[1])  # 参数b自更新

    # 每个epoch，打印loss信息
    print("Epoch {}, loss: {}".format(epoch, loss_all/4))
    train_loss_results.append(loss_all / 4)  # 将4个step的loss求平均记录在此变量中
    loss_all = 0  # loss_all归零，为记录下一个epoch的loss做准备

    # 测试部分
    # total_correct为预测对的样本个数, total_number为测试的总样本数，将这两个变量都初始化为0
    total_correct, total_number = 0, 0
    for x_test, y_test in test_db:
        # 使用更新后的参数进行预测
        y = tf.matmul(x_test, w1) + b1
        y = tf.nn.softmax(y)
        # get prediction
        pred = tf.argmax(y, axis=1)  # 返回y中最大值的索引，即预测的分类
        # 将pred转换为y_test的数据类型
        pred = tf.cast(pred, dtype=y_test.dtype)
        # 若分类正确，则correct=1，否则为0，将bool型的结果转换为int型
        correct = tf.cast(tf.equal(pred, y_test), dtype=tf.int32)
        # 将每个batch的correct数加起来
        correct = tf.reduce_sum(correct)
        # 将所有batch中的correct数加起来
        total_correct += int(correct)
        # total_number为测试的总样本数，也就是x_test的行数，shape[0]返回变量的行数
        total_number += x_test.shape[0]
    # 总的准确率等于total_correct/total_number
    acc = total_correct / total_number
    test_acc.append(acc)
    print("Test_acc:", acc)
    print("--------------------------")

# 绘制 loss 曲线
plt.title('Loss Function Curve')  # 图片标题
plt.xlabel('Epoch')  # x轴变量名称
plt.ylabel('Loss')  # y轴变量名称
plt.plot(train_loss_results, label="$Loss$")  # 逐点画出trian_loss_results值并连线，连线图标是Loss
plt.legend()  # 画出曲线图标
plt.show()  # 画出图像

# 绘制 Accuracy 曲线
plt.title('Acc Curve')  # 图片标题
plt.xlabel('Epoch')  # x轴变量名称
plt.ylabel('Acc')  # y轴变量名称
plt.plot(test_acc, label="$Accuracy$")  # 逐点画出test_acc值并连线，连线图标是Accuracy
plt.legend()
plt.show()


Epoch 0, loss: 0.282131090760231
Test_acc: 0.16666666666666666
--------------------------
Epoch 1, loss: 0.25459614396095276
Test_acc: 0.16666666666666666
--------------------------
Epoch 2, loss: 0.22570250183343887
Test_acc: 0.16666666666666666
--------------------------
Epoch 3, loss: 0.21028399094939232
Test_acc: 0.16666666666666666
--------------------------
Epoch 4, loss: 0.19942265003919601
Test_acc: 0.16666666666666666
--------------------------
Epoch 5, loss: 0.18873638287186623
Test_acc: 0.5
--------------------------
Epoch 6, loss: 0.17851299792528152
Test_acc: 0.5333333333333333
--------------------------
Epoch 7, loss: 0.16922876238822937
Test_acc: 0.5333333333333333
--------------------------
Epoch 8, loss: 0.16107673570513725
Test_acc: 0.5333333333333333
--------------------------
Epoch 9, loss: 0.15404684469103813
Test_acc: 0.5333333333333333
--------------------------
Epoch 10, loss: 0.14802725985646248
Test_acc: 0.5333333333333333
--------------------------
Epoch 11, 

2022-10-29 00:50:24.928805: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


Epoch 56, loss: 0.08750772476196289
Test_acc: 0.6
--------------------------
Epoch 57, loss: 0.08697944320738316
Test_acc: 0.6
--------------------------
Epoch 58, loss: 0.08645842969417572
Test_acc: 0.6
--------------------------
Epoch 59, loss: 0.08594448491930962
Test_acc: 0.6
--------------------------
Epoch 60, loss: 0.08543741516768932
Test_acc: 0.6
--------------------------
Epoch 61, loss: 0.08493701927363873
Test_acc: 0.6
--------------------------
Epoch 62, loss: 0.08444313332438469
Test_acc: 0.6333333333333333
--------------------------
Epoch 63, loss: 0.08395559526979923
Test_acc: 0.6333333333333333
--------------------------
Epoch 64, loss: 0.08347426168620586
Test_acc: 0.6333333333333333
--------------------------
Epoch 65, loss: 0.08299897983670235
Test_acc: 0.6333333333333333
--------------------------
Epoch 66, loss: 0.08252961747348309
Test_acc: 0.6333333333333333
--------------------------
Epoch 67, loss: 0.08206603862345219
Test_acc: 0.6333333333333333
-------------

2022-10-29 00:50:26.594859: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


Epoch 117, loss: 0.0644997050985694
Test_acc: 0.9333333333333333
--------------------------
Epoch 118, loss: 0.0642358623445034
Test_acc: 0.9333333333333333
--------------------------
Epoch 119, loss: 0.06397470459342003
Test_acc: 0.9333333333333333
--------------------------
Epoch 120, loss: 0.06371619738638401
Test_acc: 0.9333333333333333
--------------------------
Epoch 121, loss: 0.06346031371504068
Test_acc: 0.9333333333333333
--------------------------
Epoch 122, loss: 0.06320701446384192
Test_acc: 0.9333333333333333
--------------------------
Epoch 123, loss: 0.06295627448707819
Test_acc: 0.9333333333333333
--------------------------
Epoch 124, loss: 0.06270804908126593
Test_acc: 0.9333333333333333
--------------------------
Epoch 125, loss: 0.062462314032018185
Test_acc: 0.9333333333333333
--------------------------
Epoch 126, loss: 0.06221903953701258
Test_acc: 0.9333333333333333
--------------------------
Epoch 127, loss: 0.06197819020599127
Test_acc: 0.9333333333333333
-----

2022-10-29 00:50:30.441531: W tensorflow/core/data/root_dataset.cc:266] Optimization loop failed: CANCELLED: Operation was cancelled


Epoch 305, loss: 0.040056098252534866
Test_acc: 1.0
--------------------------
Epoch 306, loss: 0.03999444097280502
Test_acc: 1.0
--------------------------
Epoch 307, loss: 0.039933128748089075
Test_acc: 1.0
--------------------------
Epoch 308, loss: 0.039872155059129
Test_acc: 1.0
--------------------------
Epoch 309, loss: 0.03981153294444084
Test_acc: 1.0
--------------------------
Epoch 310, loss: 0.03975124564021826
Test_acc: 1.0
--------------------------
Epoch 311, loss: 0.039691298734396696
Test_acc: 1.0
--------------------------
Epoch 312, loss: 0.0396316796541214
Test_acc: 1.0
--------------------------
Epoch 313, loss: 0.039572392124682665
Test_acc: 1.0
--------------------------
Epoch 314, loss: 0.03951342822983861
Test_acc: 1.0
--------------------------
Epoch 315, loss: 0.039454794488847256
Test_acc: 1.0
--------------------------
Epoch 316, loss: 0.03939648065716028
Test_acc: 1.0
--------------------------
Epoch 317, loss: 0.03933848859742284
Test_acc: 1.0
-----------